CODE FOR WEB SCRAPING


In [2]:
!pip install selenium
!pip install beautifulsoup4
!pip install pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 26.1 MB/s eta 0:00:00


In [3]:
# Import necessary libraries
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

# Initialize Selenium WebDriver (you may need to specify the path to your chromedriver)
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

# URL to scrape
url = "https://www.amazon.in/s?rh=n%3A6612025031&fs=true&ref=lp_6612025031_sar"
driver.get(url)
time.sleep(3)  # Initial sleep to load the page

# Scroll to load all products (Amazon may use lazy loading)
scroll_pause_time = 2  # Time to wait after scrolling
screen_height = driver.execute_script("return window.screen.height;")  # Get screen height
scrolls = 3  # Number of scrolls, increase if you need more data

for _ in range(scrolls):
    driver.execute_script("window.scrollTo(0, window.scrollY + {});".format(screen_height))
    time.sleep(scroll_pause_time)

# Extract page source and parse with BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

# Lists to store the scraped data
product_names = []
prices = []
ratings = []
sellers = []

# Find all product containers
products = soup.find_all('div', {'data-component-type': 's-search-result'})

# Loop through each product and extract details
for product in products:
    # Extract product name
    name = product.h2.text.strip() if product.h2 else 'N/A'
    product_names.append(name)

    # Extract price
    price = product.find('span', 'a-price-whole')
    if price:
        price = price.text.strip()
    else:
        price = 'N/A'
    prices.append(price)

    # Extract rating
    rating = product.find('span', 'a-icon-alt')
    if rating:
        rating = rating.text.strip().split()[0]  # Get only rating number
    else:
        rating = 'N/A'
    ratings.append(rating)

    # Extract seller name if available (Amazon pages often lack this detail in the listings)
    seller = product.find('span', class_='a-size-base-plus')
    if seller:
        seller = seller.text.strip()
    else:
        seller = 'N/A'
    sellers.append(seller)

# Create DataFrame and save to CSV
data = {
    'Product Name': product_names,
    'Price': prices,
    'Rating': ratings,
    'Seller Name': sellers
}

df = pd.DataFrame(data)
df.to_csv('amazon_products.csv', index=False)

print("Scraping completed and saved to amazon_products.csv")


Scraping completed and saved to amazon_products.csv
